Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock
import importlib

In [3]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

# Open the text file in read mode
with open('stocks.txt', 'r') as file:
    # Read each line of the file
    for line in file:
        # Strip any leading or trailing whitespace and append the line to the list
        stocks_list.append(line.strip())

df_portfolio = pd.read_csv('config/portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

In [4]:
yf_handler = YFHandler(stock_list=['MSFT'], schemas=schemas)

In [9]:
yf_handler.stocks.tickers['MSFT'].info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

### Stocks Data

In [4]:
analysis = analyze_stock(
	ticker= ['SNOW'], 
	dic_files=dic_files,
	dic_assistants=dic_assistants
)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longe

file name: df_stocks.csv, file id: file-shEIH3Cy5PXBiE1WVS3ifFzJ has been deleted
file name: df_stocks.csv is uploaded, new file id: file-yBCEqzOTxZ6jGbnL2UEahreJ
openai_upload_files/openai_files.json file has been updated
file name: df_cashflow.csv, file id: file-rw5w6QHbRCtID1bXzrZ6JDsS has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-pr7bR9u0RiyvLzGcUSHT8nNW
openai_upload_files/openai_files.json file has been updated
file name: df_income_stmt.csv, file id: file-AYj1AdSR2DvVO1idhQ7MD9Yk has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-AJH1C2eYgjDz4ErKTVmDVn1P
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U

assistant > Here are the information gathered from the financial analyst: 

These are the stocks the client wants

In [6]:
print(analysis)

Based on the detailed financial data and historical stock prices for Snowflake Inc. (SNOW), here is a concise assessment of the stock to provide a recommendation for your client:

### Snowflake Inc. (SNOW) - Financial Analysis and Stock Recommendation

**Income Statement Overview (Last 4 Fiscal Years):**
- **Revenue Growth:** Impressive increase, with total revenue growing from $592.05 million in 2021 to $2.81 billion in 2024. This indicates robust business expansion.
- **Gross Profit:** Significant growth in gross profit, reaching $1.91 billion in 2024 from $349.46 million in 2021, reflecting an efficient revenue model.
- **EBITDA:** The company has recorded negative EBITDA consistently, with -$974.87 million in 2024. This highlights that despite the revenue growth, Snowflake is not yet profitable on an EBITDA basis.
- **Net Income:** Snowflake has posted substantial net losses, with the most recent being -$838.00 million in 2024. This continues a trend of losses, indicating high oper